In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib.figure import Figure
import math
import imageio
import datetime
from os import walk
from scipy import stats

from ConvLSTM import ConvLSTM
from SlimAutoencoder import SlimAutoencoderBuilder
from rea_cd_autoencoder import REA_CD_Autoencoder
from dataloader import hillsborough_county_rea_cd_encoded as hc_rea_cd_encoded
from datawrangler import DataWrangler


# random forest
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import mean_squared_error
import random

#file io
import pickle

import multiprocessing
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [25]:
def buildTrainingDataset():
    from os import walk
    mypath = '../Datasets/windows'
    filenames = next(walk(mypath), (None, None, []))[2]  # [] if no file
    filenames.sort(reverse=True)
    filenames = filenames[3:] #renove last 1.5 years (after the foreclosure moratorium started)
    return filenames
training_dfs = buildTrainingDataset()
print(training_dfs)

['df.2015-02-11..2020-02-10.pickle', 'df.2014-08-13..2019-08-12.pickle', 'df.2014-02-11..2019-02-10.pickle', 'df.2013-08-13..2018-08-12.pickle', 'df.2013-02-11..2018-02-10.pickle', 'df.2012-08-13..2017-08-12.pickle', 'df.2012-02-12..2017-02-10.pickle', 'df.2011-08-14..2016-08-12.pickle', 'df.2011-02-12..2016-02-11.pickle', 'df.2010-08-14..2015-08-13.pickle', 'df.2010-02-12..2015-02-11.pickle', 'df.2009-08-14..2014-08-13.pickle', 'df.2009-02-12..2014-02-11.pickle', 'df.2008-08-14..2013-08-13.pickle', 'df.2008-02-13..2013-02-11.pickle', 'df.2007-08-15..2012-08-13.pickle', 'df.2007-02-13..2012-02-12.pickle', 'df.2006-08-15..2011-08-14.pickle', 'df.2006-02-13..2011-02-12.pickle', 'df.2005-08-15..2010-08-14.pickle', 'df.2005-02-13..2010-02-12.pickle', 'df.2004-08-15..2009-08-14.pickle', 'df.2004-02-14..2009-02-12.pickle', 'df.2003-08-16..2008-08-14.pickle', 'df.2003-02-14..2008-02-13.pickle', 'df.2002-08-16..2007-08-15.pickle', 'df.2002-02-14..2007-02-13.pickle', 'df.2001-08-16..2006-08-15.

In [41]:
df = pd.read_pickle('../Datasets/residential_conveyances_jan2000_and_up_ohe.pickle')
df.shape

(1286573, 753)

In [92]:
def one_hot_encode(df,cols):
    df = df.copy()
    original_columns = df.columns
    sizes = dict()
    for col in cols:
        start_width = df.shape[1]
        df = pd.get_dummies(df, columns=[col], drop_first=False)
        sizes[col] = df.shape[1] - start_width + 1
    return df
df = one_hot_encode(df,['REA_CD'])

In [93]:
display(df[:5])
display(df[-5:])


,FOLIO,S_DATE_x,S_AMT_x,STR,tBEDS,tBATHS,tSTORIES,tUNITS,tBLDGS,JUST,LAND,BLDG,EXF,ACT,EFF,HEAT_AR,ASD_VAL,TAX_VAL,BASE,ACREAGE,NBHC_x_201001.0,NBHC_x_201002.0,NBHC_x_201003.0,NBHC_x_201004.0,NBHC_x_201005.0,NBHC_x_201006.0,NBHC_x_201007.0,NBHC_x_202001.0,NBHC_x_202002.0,NBHC_x_202003.0,NBHC_x_202004.0,NBHC_x_202005.0,NBHC_x_202006.0,NBHC_x_202007.0,NBHC_x_202008.0,NBHC_x_202009.0,NBHC_x_203001.0,NBHC_x_203002.0,NBHC_x_203003.0,NBHC_x_203004.0,NBHC_x_203005.0,NBHC_x_203006.0,NBHC_x_203007.0,NBHC_x_203008.0,NBHC_x_203009.0,NBHC_x_203010.0,NBHC_x_203011.0,NBHC_x_203012.0,NBHC_x_203013.0,NBHC_x_203014.0,NBHC_x_203015.0,NBHC_x_204001.0,NBHC_x_204002.0,NBHC_x_204003.0,NBHC_x_204004.0,NBHC_x_204005.0,NBHC_x_204006.0,NBHC_x_204007.0,NBHC_x_204008.0,NBHC_x_205001.0,NBHC_x_205002.0,NBHC_x_205003.0,NBHC_x_205004.0,NBHC_x_205005.0,NBHC_x_205006.0,NBHC_x_205007.0,NBHC_x_205008.0,NBHC_x_205009.0,NBHC_x_205010.0,NBHC_x_205011.0,NBHC_x_205012.0,NBHC_x_205013.0,NBHC_x_205014.0,NBHC_x_206001.0,NBHC_x_206002.0,NBHC_x_206003.0,NBHC_x_206004.0,NBHC_x_206005.0,NBHC_x_206006.0,NBHC_x_206007.0,NBHC_x_206008.0,NBHC_x_206009.0,NBHC_x_206010.0,NBHC_x_206011.0,NBHC_x_206012.0,NBHC_x_206013.0,NBHC_x_206014.0,NBHC_x_206015.0,NBHC_x_206016.0,NBHC_x_206017.0,NBHC_x_206018.0,NBHC_x_206019.0,NBHC_x_206020.0,NBHC_x_207001.0,NBHC_x_207002.0,NBHC_x_207003.0,NBHC_x_207004.0,NBHC_x_207005.0,NBHC_x_207006.0,NBHC_x_208001.0,NBHC_x_208002.0,NBHC_x_208003.0,NBHC_x_208004.0,NBHC_x_208005.0,NBHC_x_208006.0,NBHC_x_208007.0,NBHC_x_208008.0,NBHC_x_208009.0,NBHC_x_208010.0,NBHC_x_209001.0,NBHC_x_209002.0,NBHC_x_209003.0,NBHC_x_209004.0,NBHC_x_209005.0,NBHC_x_209006.0,NBHC_x_209007.0,NBHC_x_209008.0,NBHC_x_209009.0,NBHC_x_209010.0,NBHC_x_209012.0,NBHC_x_209013.0,NBHC_x_209014.0,NBHC_x_209015.0,NBHC_x_209016.0,NBHC_x_210001.0,NBHC_x_210002.0,NBHC_x_210003.0,NBHC_x_210004.0,NBHC_x_210005.0,NBHC_x_210006.0,NBHC_x_210007.0,NBHC_x_210008.0,NBHC_x_210009.0,NBHC_x_210010.0,NBHC_x_210011.0,NBHC_x_210012.0,NBHC_x_210013.0,NBHC_x_210014.0,NBHC_x_210015.0,NBHC_x_210016.0,NBHC_x_211001.0,NBHC_x_211002.0,NBHC_x_211003.0,NBHC_x_211004.0,NBHC_x_211005.0,NBHC_x_211006.0,NBHC_x_211007.0,NBHC_x_211008.0,NBHC_x_211009.0,NBHC_x_211010.0,NBHC_x_211011.0,NBHC_x_212001.0,NBHC_x_212002.0,NBHC_x_212003.0,NBHC_x_212004.0,NBHC_x_212005.0,NBHC_x_212006.0,NBHC_x_212007.0,NBHC_x_212008.0,NBHC_x_212009.0,NBHC_x_212010.0,NBHC_x_213001.0,NBHC_x_213002.0,NBHC_x_213003.0,NBHC_x_213004.0,NBHC_x_213005.0,NBHC_x_213006.0,NBHC_x_213007.0,NBHC_x_214001.0,NBHC_x_214002.0,NBHC_x_214003.0,NBHC_x_214004.0,NBHC_x_214005.0,NBHC_x_214006.0,NBHC_x_214007.0,NBHC_x_214008.0,NBHC_x_214009.0,NBHC_x_215001.0,NBHC_x_215002.0,NBHC_x_215003.0,NBHC_x_215004.0,NBHC_x_215005.0,NBHC_x_215006.0,NBHC_x_215007.0,NBHC_x_215008.0,NBHC_x_215009.0,NBHC_x_215010.0,NBHC_x_215011.0,NBHC_x_215012.0,NBHC_x_215013.0,NBHC_x_215014.0,NBHC_x_215016.0,NBHC_x_215017.0,NBHC_x_215018.0,NBHC_x_215019.0,NBHC_x_215020.0,NBHC_x_215021.0,NBHC_x_215022.0,NBHC_x_215023.0,NBHC_x_216001.0,NBHC_x_216002.0,NBHC_x_216003.0,NBHC_x_216004.0,NBHC_x_216005.0,NBHC_x_216006.0,NBHC_x_216007.0,NBHC_x_216008.0,NBHC_x_216009.0,NBHC_x_216010.0,NBHC_x_216011.0,NBHC_x_216012.0,NBHC_x_217001.0,NBHC_x_217002.0,NBHC_x_217003.0,NBHC_x_217004.0,NBHC_x_217005.0,NBHC_x_217006.0,NBHC_x_217007.0,NBHC_x_218001.0,NBHC_x_218002.0,NBHC_x_218003.0,NBHC_x_218004.0,NBHC_x_218005.0,NBHC_x_218006.0,NBHC_x_218007.0,NBHC_x_218008.0,NBHC_x_218009.0,NBHC_x_218010.0,NBHC_x_218011.0,NBHC_x_219001.0,NBHC_x_219002.0,NBHC_x_219003.0,NBHC_x_219004.0,NBHC_x_219005.0,NBHC_x_219006.0,NBHC_x_219007.0,NBHC_x_220001.0,NBHC_x_220002.0,NBHC_x_220003.0,NBHC_x_220004.0,NBHC_x_220005.0,NBHC_x_220006.0,NBHC_x_220007.0,NBHC_x_220008.0,NBHC_x_220009.0,NBHC_x_220010.0,NBHC_x_220011.0,NBHC_x_220012.0,NBHC_x_220013.0,NBHC_x_220014.0,NBHC_x_220015.0,NBHC_x_220016.0,NBHC_x_221001.0,NBHC_x_221002.0,NBHC_x_221003.0,NBHC_x_221004.0,NBHC_x_221005.0,NBHC_x_221006.0,NBHC_x_221007.0,NBHC_x_221008.0,NBHC_x_221009.0,NBHC_x_221010.0,

,FOLIO,S_DATE_x,S_AMT_x,STR,tBEDS,tBATHS,tSTORIES,tUNITS,tBLDGS,JUST,LAND,BLDG,EXF,ACT,EFF,HEAT_AR,ASD_VAL,TAX_VAL,BASE,ACREAGE,NBHC_x_201001.0,NBHC_x_201002.0,NBHC_x_201003.0,NBHC_x_201004.0,NBHC_x_201005.0,NBHC_x_201006.0,NBHC_x_201007.0,NBHC_x_202001.0,NBHC_x_202002.0,NBHC_x_202003.0,NBHC_x_202004.0,NBHC_x_202005.0,NBHC_x_202006.0,NBHC_x_202007.0,NBHC_x_202008.0,NBHC_x_202009.0,NBHC_x_203001.0,NBHC_x_203002.0,NBHC_x_203003.0,NBHC_x_203004.0,NBHC_x_203005.0,NBHC_x_203006.0,NBHC_x_203007.0,NBHC_x_203008.0,NBHC_x_203009.0,NBHC_x_203010.0,NBHC_x_203011.0,NBHC_x_203012.0,NBHC_x_203013.0,NBHC_x_203014.0,NBHC_x_203015.0,NBHC_x_204001.0,NBHC_x_204002.0,NBHC_x_204003.0,NBHC_x_204004.0,NBHC_x_204005.0,NBHC_x_204006.0,NBHC_x_204007.0,NBHC_x_204008.0,NBHC_x_205001.0,NBHC_x_205002.0,NBHC_x_205003.0,NBHC_x_205004.0,NBHC_x_205005.0,NBHC_x_205006.0,NBHC_x_205007.0,NBHC_x_205008.0,NBHC_x_205009.0,NBHC_x_205010.0,NBHC_x_205011.0,NBHC_x_205012.0,NBHC_x_205013.0,NBHC_x_205014.0,NBHC_x_206001.0,NBHC_x_206002.0,NBHC_x_206003.0,NBHC_x_206004.0,NBHC_x_206005.0,NBHC_x_206006.0,NBHC_x_206007.0,NBHC_x_206008.0,NBHC_x_206009.0,NBHC_x_206010.0,NBHC_x_206011.0,NBHC_x_206012.0,NBHC_x_206013.0,NBHC_x_206014.0,NBHC_x_206015.0,NBHC_x_206016.0,NBHC_x_206017.0,NBHC_x_206018.0,NBHC_x_206019.0,NBHC_x_206020.0,NBHC_x_207001.0,NBHC_x_207002.0,NBHC_x_207003.0,NBHC_x_207004.0,NBHC_x_207005.0,NBHC_x_207006.0,NBHC_x_208001.0,NBHC_x_208002.0,NBHC_x_208003.0,NBHC_x_208004.0,NBHC_x_208005.0,NBHC_x_208006.0,NBHC_x_208007.0,NBHC_x_208008.0,NBHC_x_208009.0,NBHC_x_208010.0,NBHC_x_209001.0,NBHC_x_209002.0,NBHC_x_209003.0,NBHC_x_209004.0,NBHC_x_209005.0,NBHC_x_209006.0,NBHC_x_209007.0,NBHC_x_209008.0,NBHC_x_209009.0,NBHC_x_209010.0,NBHC_x_209012.0,NBHC_x_209013.0,NBHC_x_209014.0,NBHC_x_209015.0,NBHC_x_209016.0,NBHC_x_210001.0,NBHC_x_210002.0,NBHC_x_210003.0,NBHC_x_210004.0,NBHC_x_210005.0,NBHC_x_210006.0,NBHC_x_210007.0,NBHC_x_210008.0,NBHC_x_210009.0,NBHC_x_210010.0,NBHC_x_210011.0,NBHC_x_210012.0,NBHC_x_210013.0,NBHC_x_210014.0,NBHC_x_210015.0,NBHC_x_210016.0,NBHC_x_211001.0,NBHC_x_211002.0,NBHC_x_211003.0,NBHC_x_211004.0,NBHC_x_211005.0,NBHC_x_211006.0,NBHC_x_211007.0,NBHC_x_211008.0,NBHC_x_211009.0,NBHC_x_211010.0,NBHC_x_211011.0,NBHC_x_212001.0,NBHC_x_212002.0,NBHC_x_212003.0,NBHC_x_212004.0,NBHC_x_212005.0,NBHC_x_212006.0,NBHC_x_212007.0,NBHC_x_212008.0,NBHC_x_212009.0,NBHC_x_212010.0,NBHC_x_213001.0,NBHC_x_213002.0,NBHC_x_213003.0,NBHC_x_213004.0,NBHC_x_213005.0,NBHC_x_213006.0,NBHC_x_213007.0,NBHC_x_214001.0,NBHC_x_214002.0,NBHC_x_214003.0,NBHC_x_214004.0,NBHC_x_214005.0,NBHC_x_214006.0,NBHC_x_214007.0,NBHC_x_214008.0,NBHC_x_214009.0,NBHC_x_215001.0,NBHC_x_215002.0,NBHC_x_215003.0,NBHC_x_215004.0,NBHC_x_215005.0,NBHC_x_215006.0,NBHC_x_215007.0,NBHC_x_215008.0,NBHC_x_215009.0,NBHC_x_215010.0,NBHC_x_215011.0,NBHC_x_215012.0,NBHC_x_215013.0,NBHC_x_215014.0,NBHC_x_215016.0,NBHC_x_215017.0,NBHC_x_215018.0,NBHC_x_215019.0,NBHC_x_215020.0,NBHC_x_215021.0,NBHC_x_215022.0,NBHC_x_215023.0,NBHC_x_216001.0,NBHC_x_216002.0,NBHC_x_216003.0,NBHC_x_216004.0,NBHC_x_216005.0,NBHC_x_216006.0,NBHC_x_216007.0,NBHC_x_216008.0,NBHC_x_216009.0,NBHC_x_216010.0,NBHC_x_216011.0,NBHC_x_216012.0,NBHC_x_217001.0,NBHC_x_217002.0,NBHC_x_217003.0,NBHC_x_217004.0,NBHC_x_217005.0,NBHC_x_217006.0,NBHC_x_217007.0,NBHC_x_218001.0,NBHC_x_218002.0,NBHC_x_218003.0,NBHC_x_218004.0,NBHC_x_218005.0,NBHC_x_218006.0,NBHC_x_218007.0,NBHC_x_218008.0,NBHC_x_218009.0,NBHC_x_218010.0,NBHC_x_218011.0,NBHC_x_219001.0,NBHC_x_219002.0,NBHC_x_219003.0,NBHC_x_219004.0,NBHC_x_219005.0,NBHC_x_219006.0,NBHC_x_219007.0,NBHC_x_220001.0,NBHC_x_220002.0,NBHC_x_220003.0,NBHC_x_220004.0,NBHC_x_220005.0,NBHC_x_220006.0,NBHC_x_220007.0,NBHC_x_220008.0,NBHC_x_220009.0,NBHC_x_220010.0,NBHC_x_220011.0,NBHC_x_220012.0,NBHC_x_220013.0,NBHC_x_220014.0,NBHC_x_220015.0,NBHC_x_220016.0,NBHC_x_221001.0,NBHC_x_221002.0,NBHC_x_221003.0,NBHC_x_221004.0,NBHC_x_221005.0,NBHC_x_221006.0,NBHC_x_221007.0,NBHC_x_221008.0,NBHC_x_221009.0,NBHC_x_221010.0,

In [103]:
def adjustDates(df, date_col, new_col_name, new_end_date):
    #print(new_end_date[0], type(new_end_date[0]))
    out = df.copy()
    out[new_col_name] = (new_end_date - pd.to_datetime(df[date_col])).dt.days
    return out

# Find foreclosures in our 6-month windows
def FindForeclosures(df, end_date, start_date='2001-01-01', history_length_in_days=365*10, step_size_in_days=365/2, nonforeclosured_folio_list):
    '''
    history_length_in_days - the number of days to look back and generate a history for
    step_size_in_days      - the number of days we step backwards to generate a new history
    '''
    df = df.copy()
    
    
    
    current_end_date = pd.to_datetime(end_date)
    #current_end_date = end_date
    start_date = pd.to_datetime(start_date)
    print(type(current_end_date))
    current_start_date = pd.to_datetime(current_end_date) - pd.to_timedelta(f"{step_size_in_days} days")
    history_start_date = pd.to_datetime(current_end_date) - pd.to_timedelta(f"{history_length_in_days} days")
    window = 0
    print(f"counting backwards from {end_date}")
    uber_folio_history = None
    uber_nonforeclosure_history = None
    nonforeclosed_folio_list_used = set()
    foreclosed_folios = []
    while history_start_date >= start_date:
        if len(foreclosed_folios) > 0:
            history_start_date = pd.to_datetime(current_end_date) - pd.to_timedelta(f"{history_length_in_days} days")
            test_folios = 0
            num_folios_added = 0
            #for folio in foreclosed_folios:
            print(f"{history_start_date} to {current_end_date}")
            
            
            # get foreclosed folios
            folio_history = df[ df['FOLIO'].isin(foreclosed_folios) & (pd.to_datetime(df['S_DATE_x']) >= history_start_date) & (pd.to_datetime(df['S_DATE_x']) < current_end_date) ]
            if folio_history.shape[0] <= 0:
                continue
            folio_history = adjustDates(folio_history, 'S_DATE_x', 'RecordAge', current_end_date)
            folio_history = adjustDates(folio_history, 'ACT', 'BuiltDaysAgo', current_end_date)
            folio_history = adjustDates(folio_history, 'EFF', 'DepreciationAge', current_end_date)
            folio_history['foreclose_in_six_months'] = 1
            folio_history['startdate'] = history_start_date
            folio_history['enddate'] = current_end_date

            if uber_folio_history is None:
                uber_folio_history = folio_history.copy()
            else: 
                uber_folio_history.append( folio_history )

            num_new_folios = folio_history['FOLIO'].unique().shape[0]
            
            ## get an equivalent number of non-foreclosed folios
            new_nonforeclosed_folios = np.random.choice( df[ df['FOLIO'].isin(nonforeclosed_folios_list), size=num_new_folios, replace=False)
            nonforeclosed_folio_history = df[ df['FOLIO'].isin(foreclosed_folios) & (pd.to_datetime(df['S_DATE_x']) >= history_start_date) & (pd.to_datetime(df['S_DATE_x']) < current_end_date) ]
            
#            display(folio_history)

            
            
            #select an equal number of never-foreclosed properties
            
        #do logic
        foreclosures = df[ (df['S_TYPE_CT'] == 1) & (pd.to_datetime(df['S_DATE_x']) >= current_start_date) & (pd.to_datetime(df['S_DATE_x']) < current_end_date) ]
        foreclosed_folios = foreclosures['FOLIO'].unique()
        #print(f"{current_start_date.strftime('%Y-%m-%d')[0]} to {current_end_date.strftime('%Y-%m-%d')[0]} {foreclosures.shape}")
        #print(f"{current_start_date.strftime('%Y-%m-%d')} to {current_end_date.strftime('%Y-%m-%d')} {foreclosures.shape}")
        
        #find foreclosures
        
    
        #update search dates
        current_end_date = pd.to_datetime(current_end_date) - pd.to_timedelta(f"{step_size_in_days} days")
        current_start_date = pd.to_datetime(current_end_date) - pd.to_timedelta(f"{step_size_in_days} days")
        continue
    return uber_folio_history

        
allwindows_foreclosures = FindForeclosures(df, end_date='2020-02-10')

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
counting backwards from 2020-02-10
2009-08-13 12:00:00 to 2019-08-11 12:00:00
2009-02-12 00:00:00 to 2019-02-10 00:00:00
2008-08-13 12:00:00 to 2018-08-11 12:00:00
2008-02-13 00:00:00 to 2018-02-10 00:00:00
2007-08-14 12:00:00 to 2017-08-11 12:00:00
2007-02-13 00:00:00 to 2017-02-10 00:00:00
2006-08-14 12:00:00 to 2016-08-11 12:00:00
2006-02-13 00:00:00 to 2016-02-11 00:00:00
2005-08-14 12:00:00 to 2015-08-12 12:00:00
2005-02-13 00:00:00 to 2015-02-11 00:00:00
2004-08-14 12:00:00 to 2014-08-12 12:00:00
2004-02-14 00:00:00 to 2014-02-11 00:00:00
2003-08-15 12:00:00 to 2013-08-12 12:00:00
2003-02-14 00:00:00 to 2013-02-11 00:00:00
2002-08-15 12:00:00 to 2012-08-12 12:00:00
2002-02-14 00:00:00 to 2012-02-12 00:00:00
2001-08-15 12:00:00 to 2011-08-13 12:00:00
2001-02-14 00:00:00 to 2011-02-12 00:00:00
2000-08-15 12:00:00 to 2010-08-13 12:00:00


In [104]:
allwindows_foreclosures.to_pickle('../Datasets/windows/allwindows_foreclosures.pickle')
allwindows_foreclosures.head()

,FOLIO,S_DATE_x,S_AMT_x,STR,tBEDS,tBATHS,tSTORIES,tUNITS,tBLDGS,JUST,LAND,BLDG,EXF,ACT,EFF,HEAT_AR,ASD_VAL,TAX_VAL,BASE,ACREAGE,NBHC_x_201001.0,NBHC_x_201002.0,NBHC_x_201003.0,NBHC_x_201004.0,NBHC_x_201005.0,NBHC_x_201006.0,NBHC_x_201007.0,NBHC_x_202001.0,NBHC_x_202002.0,NBHC_x_202003.0,NBHC_x_202004.0,NBHC_x_202005.0,NBHC_x_202006.0,NBHC_x_202007.0,NBHC_x_202008.0,NBHC_x_202009.0,NBHC_x_203001.0,NBHC_x_203002.0,NBHC_x_203003.0,NBHC_x_203004.0,NBHC_x_203005.0,NBHC_x_203006.0,NBHC_x_203007.0,NBHC_x_203008.0,NBHC_x_203009.0,NBHC_x_203010.0,NBHC_x_203011.0,NBHC_x_203012.0,NBHC_x_203013.0,NBHC_x_203014.0,NBHC_x_203015.0,NBHC_x_204001.0,NBHC_x_204002.0,NBHC_x_204003.0,NBHC_x_204004.0,NBHC_x_204005.0,NBHC_x_204006.0,NBHC_x_204007.0,NBHC_x_204008.0,NBHC_x_205001.0,NBHC_x_205002.0,NBHC_x_205003.0,NBHC_x_205004.0,NBHC_x_205005.0,NBHC_x_205006.0,NBHC_x_205007.0,NBHC_x_205008.0,NBHC_x_205009.0,NBHC_x_205010.0,NBHC_x_205011.0,NBHC_x_205012.0,NBHC_x_205013.0,NBHC_x_205014.0,NBHC_x_206001.0,NBHC_x_206002.0,NBHC_x_206003.0,NBHC_x_206004.0,NBHC_x_206005.0,NBHC_x_206006.0,NBHC_x_206007.0,NBHC_x_206008.0,NBHC_x_206009.0,NBHC_x_206010.0,NBHC_x_206011.0,NBHC_x_206012.0,NBHC_x_206013.0,NBHC_x_206014.0,NBHC_x_206015.0,NBHC_x_206016.0,NBHC_x_206017.0,NBHC_x_206018.0,NBHC_x_206019.0,NBHC_x_206020.0,NBHC_x_207001.0,NBHC_x_207002.0,NBHC_x_207003.0,NBHC_x_207004.0,NBHC_x_207005.0,NBHC_x_207006.0,NBHC_x_208001.0,NBHC_x_208002.0,NBHC_x_208003.0,NBHC_x_208004.0,NBHC_x_208005.0,NBHC_x_208006.0,NBHC_x_208007.0,NBHC_x_208008.0,NBHC_x_208009.0,NBHC_x_208010.0,NBHC_x_209001.0,NBHC_x_209002.0,NBHC_x_209003.0,NBHC_x_209004.0,NBHC_x_209005.0,NBHC_x_209006.0,NBHC_x_209007.0,NBHC_x_209008.0,NBHC_x_209009.0,NBHC_x_209010.0,NBHC_x_209012.0,NBHC_x_209013.0,NBHC_x_209014.0,NBHC_x_209015.0,NBHC_x_209016.0,NBHC_x_210001.0,NBHC_x_210002.0,NBHC_x_210003.0,NBHC_x_210004.0,NBHC_x_210005.0,NBHC_x_210006.0,NBHC_x_210007.0,NBHC_x_210008.0,NBHC_x_210009.0,NBHC_x_210010.0,NBHC_x_210011.0,NBHC_x_210012.0,NBHC_x_210013.0,NBHC_x_210014.0,NBHC_x_210015.0,NBHC_x_210016.0,NBHC_x_211001.0,NBHC_x_211002.0,NBHC_x_211003.0,NBHC_x_211004.0,NBHC_x_211005.0,NBHC_x_211006.0,NBHC_x_211007.0,NBHC_x_211008.0,NBHC_x_211009.0,NBHC_x_211010.0,NBHC_x_211011.0,NBHC_x_212001.0,NBHC_x_212002.0,NBHC_x_212003.0,NBHC_x_212004.0,NBHC_x_212005.0,NBHC_x_212006.0,NBHC_x_212007.0,NBHC_x_212008.0,NBHC_x_212009.0,NBHC_x_212010.0,NBHC_x_213001.0,NBHC_x_213002.0,NBHC_x_213003.0,NBHC_x_213004.0,NBHC_x_213005.0,NBHC_x_213006.0,NBHC_x_213007.0,NBHC_x_214001.0,NBHC_x_214002.0,NBHC_x_214003.0,NBHC_x_214004.0,NBHC_x_214005.0,NBHC_x_214006.0,NBHC_x_214007.0,NBHC_x_214008.0,NBHC_x_214009.0,NBHC_x_215001.0,NBHC_x_215002.0,NBHC_x_215003.0,NBHC_x_215004.0,NBHC_x_215005.0,NBHC_x_215006.0,NBHC_x_215007.0,NBHC_x_215008.0,NBHC_x_215009.0,NBHC_x_215010.0,NBHC_x_215011.0,NBHC_x_215012.0,NBHC_x_215013.0,NBHC_x_215014.0,NBHC_x_215016.0,NBHC_x_215017.0,NBHC_x_215018.0,NBHC_x_215019.0,NBHC_x_215020.0,NBHC_x_215021.0,NBHC_x_215022.0,NBHC_x_215023.0,NBHC_x_216001.0,NBHC_x_216002.0,NBHC_x_216003.0,NBHC_x_216004.0,NBHC_x_216005.0,NBHC_x_216006.0,NBHC_x_216007.0,NBHC_x_216008.0,NBHC_x_216009.0,NBHC_x_216010.0,NBHC_x_216011.0,NBHC_x_216012.0,NBHC_x_217001.0,NBHC_x_217002.0,NBHC_x_217003.0,NBHC_x_217004.0,NBHC_x_217005.0,NBHC_x_217006.0,NBHC_x_217007.0,NBHC_x_218001.0,NBHC_x_218002.0,NBHC_x_218003.0,NBHC_x_218004.0,NBHC_x_218005.0,NBHC_x_218006.0,NBHC_x_218007.0,NBHC_x_218008.0,NBHC_x_218009.0,NBHC_x_218010.0,NBHC_x_218011.0,NBHC_x_219001.0,NBHC_x_219002.0,NBHC_x_219003.0,NBHC_x_219004.0,NBHC_x_219005.0,NBHC_x_219006.0,NBHC_x_219007.0,NBHC_x_220001.0,NBHC_x_220002.0,NBHC_x_220003.0,NBHC_x_220004.0,NBHC_x_220005.0,NBHC_x_220006.0,NBHC_x_220007.0,NBHC_x_220008.0,NBHC_x_220009.0,NBHC_x_220010.0,NBHC_x_220011.0,NBHC_x_220012.0,NBHC_x_220013.0,NBHC_x_220014.0,NBHC_x_220015.0,NBHC_x_220016.0,NBHC_x_221001.0,NBHC_x_221002.0,NBHC_x_221003.0,NBHC_x_221004.0,NBHC_x_221005.0,NBHC_x_221006.0,NBHC_x_221007.0,NBHC_x_221008.0,NBHC_x_221009.0,NBHC_x_221010.0,

In [95]:
folios = first_window['FOLIO'].unique()


In [98]:
display(first_window[ first_window['FOLIO'] == np.random.choice(folios,size=1)[0]])

,FOLIO,S_DATE_x,S_AMT_x,STR,tBEDS,tBATHS,tSTORIES,tUNITS,tBLDGS,JUST,LAND,BLDG,EXF,ACT,EFF,HEAT_AR,ASD_VAL,TAX_VAL,BASE,ACREAGE,NBHC_x_201001.0,NBHC_x_201002.0,NBHC_x_201003.0,NBHC_x_201004.0,NBHC_x_201005.0,NBHC_x_201006.0,NBHC_x_201007.0,NBHC_x_202001.0,NBHC_x_202002.0,NBHC_x_202003.0,NBHC_x_202004.0,NBHC_x_202005.0,NBHC_x_202006.0,NBHC_x_202007.0,NBHC_x_202008.0,NBHC_x_202009.0,NBHC_x_203001.0,NBHC_x_203002.0,NBHC_x_203003.0,NBHC_x_203004.0,NBHC_x_203005.0,NBHC_x_203006.0,NBHC_x_203007.0,NBHC_x_203008.0,NBHC_x_203009.0,NBHC_x_203010.0,NBHC_x_203011.0,NBHC_x_203012.0,NBHC_x_203013.0,NBHC_x_203014.0,NBHC_x_203015.0,NBHC_x_204001.0,NBHC_x_204002.0,NBHC_x_204003.0,NBHC_x_204004.0,NBHC_x_204005.0,NBHC_x_204006.0,NBHC_x_204007.0,NBHC_x_204008.0,NBHC_x_205001.0,NBHC_x_205002.0,NBHC_x_205003.0,NBHC_x_205004.0,NBHC_x_205005.0,NBHC_x_205006.0,NBHC_x_205007.0,NBHC_x_205008.0,NBHC_x_205009.0,NBHC_x_205010.0,NBHC_x_205011.0,NBHC_x_205012.0,NBHC_x_205013.0,NBHC_x_205014.0,NBHC_x_206001.0,NBHC_x_206002.0,NBHC_x_206003.0,NBHC_x_206004.0,NBHC_x_206005.0,NBHC_x_206006.0,NBHC_x_206007.0,NBHC_x_206008.0,NBHC_x_206009.0,NBHC_x_206010.0,NBHC_x_206011.0,NBHC_x_206012.0,NBHC_x_206013.0,NBHC_x_206014.0,NBHC_x_206015.0,NBHC_x_206016.0,NBHC_x_206017.0,NBHC_x_206018.0,NBHC_x_206019.0,NBHC_x_206020.0,NBHC_x_207001.0,NBHC_x_207002.0,NBHC_x_207003.0,NBHC_x_207004.0,NBHC_x_207005.0,NBHC_x_207006.0,NBHC_x_208001.0,NBHC_x_208002.0,NBHC_x_208003.0,NBHC_x_208004.0,NBHC_x_208005.0,NBHC_x_208006.0,NBHC_x_208007.0,NBHC_x_208008.0,NBHC_x_208009.0,NBHC_x_208010.0,NBHC_x_209001.0,NBHC_x_209002.0,NBHC_x_209003.0,NBHC_x_209004.0,NBHC_x_209005.0,NBHC_x_209006.0,NBHC_x_209007.0,NBHC_x_209008.0,NBHC_x_209009.0,NBHC_x_209010.0,NBHC_x_209012.0,NBHC_x_209013.0,NBHC_x_209014.0,NBHC_x_209015.0,NBHC_x_209016.0,NBHC_x_210001.0,NBHC_x_210002.0,NBHC_x_210003.0,NBHC_x_210004.0,NBHC_x_210005.0,NBHC_x_210006.0,NBHC_x_210007.0,NBHC_x_210008.0,NBHC_x_210009.0,NBHC_x_210010.0,NBHC_x_210011.0,NBHC_x_210012.0,NBHC_x_210013.0,NBHC_x_210014.0,NBHC_x_210015.0,NBHC_x_210016.0,NBHC_x_211001.0,NBHC_x_211002.0,NBHC_x_211003.0,NBHC_x_211004.0,NBHC_x_211005.0,NBHC_x_211006.0,NBHC_x_211007.0,NBHC_x_211008.0,NBHC_x_211009.0,NBHC_x_211010.0,NBHC_x_211011.0,NBHC_x_212001.0,NBHC_x_212002.0,NBHC_x_212003.0,NBHC_x_212004.0,NBHC_x_212005.0,NBHC_x_212006.0,NBHC_x_212007.0,NBHC_x_212008.0,NBHC_x_212009.0,NBHC_x_212010.0,NBHC_x_213001.0,NBHC_x_213002.0,NBHC_x_213003.0,NBHC_x_213004.0,NBHC_x_213005.0,NBHC_x_213006.0,NBHC_x_213007.0,NBHC_x_214001.0,NBHC_x_214002.0,NBHC_x_214003.0,NBHC_x_214004.0,NBHC_x_214005.0,NBHC_x_214006.0,NBHC_x_214007.0,NBHC_x_214008.0,NBHC_x_214009.0,NBHC_x_215001.0,NBHC_x_215002.0,NBHC_x_215003.0,NBHC_x_215004.0,NBHC_x_215005.0,NBHC_x_215006.0,NBHC_x_215007.0,NBHC_x_215008.0,NBHC_x_215009.0,NBHC_x_215010.0,NBHC_x_215011.0,NBHC_x_215012.0,NBHC_x_215013.0,NBHC_x_215014.0,NBHC_x_215016.0,NBHC_x_215017.0,NBHC_x_215018.0,NBHC_x_215019.0,NBHC_x_215020.0,NBHC_x_215021.0,NBHC_x_215022.0,NBHC_x_215023.0,NBHC_x_216001.0,NBHC_x_216002.0,NBHC_x_216003.0,NBHC_x_216004.0,NBHC_x_216005.0,NBHC_x_216006.0,NBHC_x_216007.0,NBHC_x_216008.0,NBHC_x_216009.0,NBHC_x_216010.0,NBHC_x_216011.0,NBHC_x_216012.0,NBHC_x_217001.0,NBHC_x_217002.0,NBHC_x_217003.0,NBHC_x_217004.0,NBHC_x_217005.0,NBHC_x_217006.0,NBHC_x_217007.0,NBHC_x_218001.0,NBHC_x_218002.0,NBHC_x_218003.0,NBHC_x_218004.0,NBHC_x_218005.0,NBHC_x_218006.0,NBHC_x_218007.0,NBHC_x_218008.0,NBHC_x_218009.0,NBHC_x_218010.0,NBHC_x_218011.0,NBHC_x_219001.0,NBHC_x_219002.0,NBHC_x_219003.0,NBHC_x_219004.0,NBHC_x_219005.0,NBHC_x_219006.0,NBHC_x_219007.0,NBHC_x_220001.0,NBHC_x_220002.0,NBHC_x_220003.0,NBHC_x_220004.0,NBHC_x_220005.0,NBHC_x_220006.0,NBHC_x_220007.0,NBHC_x_220008.0,NBHC_x_220009.0,NBHC_x_220010.0,NBHC_x_220011.0,NBHC_x_220012.0,NBHC_x_220013.0,NBHC_x_220014.0,NBHC_x_220015.0,NBHC_x_220016.0,NBHC_x_221001.0,NBHC_x_221002.0,NBHC_x_221003.0,NBHC_x_221004.0,NBHC_x_221005.0,NBHC_x_221006.0,NBHC_x_221007.0,NBHC_x_221008.0,NBHC_x_221009.0,NBHC_x_221010.0,

In [21]:
for df_filename in training_dfs:
    print(f"loading {df_filename}")
    df = pd.read_pickle(f'../Datasets/windows/{df_filename}')
    print(df.shape)
    break

loading df.2015-02-11..2020-02-10.pickle
(319111, 754)


In [105]:
with open('../Datasets/notforeclosed_folios.pickle', 'rb') as f:
    notforeclosed_folios = pickle.load(f)
len(notforeclosed_folios)

334300